### IS - Brand position
Finding Product position on Social Network (PPSN)

In [1]:
import networkx as nx
import numpy as np
import pandas as pd
import itertools
import math
import string
import json
import pymongo
from itertools import groupby

In [2]:
import gensim
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=pd.core.common.SettingWithCopyWarning)

C:\ProgramData\Anaconda3\lib\site-packages\past\types\oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable
C:\ProgramData\Anaconda3\lib\site-packages\past\builtins\misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Mapping


In [3]:
import time
from datetime import datetime
today = datetime.now()

In [4]:
#!pip install pythainlp
import pythainlp
from pythainlp import Tokenizer
from pythainlp.util import normalize
from pythainlp.tokenize import word_tokenize
from pythainlp.corpus.common import thai_words
from pythainlp import sent_tokenize
from pythainlp.corpus import thai_stopwords
from pythainlp.util import dict_trie
from pythainlp.tag import pos_tag

In [5]:
# https://thainlp.org/pythainlp/docs/2.0/api/tag.html
from pythainlp.spell import correct
from pythainlp.tag import pos_tag

#https://thainlp.org/pythainlp/docs/2.0/api/transliterate.html
from pythainlp.transliterate import romanize

In [6]:
import statistics as stat
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import jaccard_score

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [7]:
# Files definition
root_input_path = 'Datasource'
root_output_path = 'output'

#comment_file = []
#comment_file.append(root_input_path+"/comment_meiji.csv")
#comment_file.append(root_input_path+"/comment_DDCF.csv")

#comment_file = []
#comment_file.append("comment_meiji.csv")
#comment_file.append("comment_DDCF.csv")

comment_nlp_file = root_output_path+"/comment_nlpToken.csv"
comment_cooc_freq_file = root_output_path+"/comment_cooc_freq.xlsx"
comment_cooc_lift_file = root_output_path+"/comment_cooc_Lift.xlsx"
comment_cooc_jacc_file = root_output_path+"/comment_cooc_jacc.xlsx"

#comment_nlp_file = "comment_nlpToken.csv"
#comment_cooc_nlp_file = "comment_cooc.xlsx"

In [8]:
# Connect MongoDB
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["NIDA_PPSN_PRD"]
col_thread = mydb["NIDA_PPSN_THREAD"]
col_comment = mydb["NIDA_PPSN_COMMENT"]
col_scrape = mydb["NIDA_PPSN_SCRAPE"]

### 1.Clean data & NLP Tokenized

In [9]:
def checkStopwords(w):
    returnList = []
    stopwords = set(thai_stopwords())
    stopwords.remove('ไม่')
    stopwords.remove('ได้')
    stopwords.remove('สูง')
    for i in w:  # comment
        if i not in stopwords:
            returnList.append(i)
    return returnList

In [10]:
def normThai(w):
    returnList = []
    for i in w:
        returnList.append(normalize(i).lower())
    return returnList

In [11]:
list_punc = []
for c in string.punctuation:
    list_punc.append(c)

In [12]:
cursor_corpus = col_scrape.find()
df_Corpus = pd.DataFrame(cursor_corpus)

In [13]:
df_Corpus.head()

,_id,URLs,headline,text,article_date,Retrived_date
0,5fc2853c77db91fa3d980f8d,https://pantip.com/topic/39868603,นมเมจิรสหวานทำไมหายากจังคะ,เมื่อก่อนหลายปีมาแล้ว ในเซเว่นยังมีนมเมจิรสหวา...,2020-05-05 13:22:59,2020-11-29 00:13:26.571
1,5fc2853c77db91fa3d980f8e,https://pantip.com/topic/39868603,นมเมจิรสหวานทำไมหายากจังคะ,เบาหวานครับ แฮ่ๆ,2020-05-05 13:22:59,2020-11-29 00:13:26.571
2,5fc2853c77db91fa3d980f8f,https://pantip.com/topic/39868603,นมเมจิรสหวานทำไมหายากจังคะ,หาซื้อไม่ได้นี่คิดได้ 2 อย่างนะ. คนไม่ค่อย...,2020-05-05 13:22:59,2020-11-29 00:13:26.571
3,5fc2853c77db91fa3d980f90,https://pantip.com/topic/39868603,นมเมจิรสหวานทำไมหายากจังคะ,เป็นรสที่ขายไม่ดีครับ\nรสพื้นฐานคือ\nจืด แบ่งย...,2020-05-05 13:22:59,2020-11-29 00:13:26.571
4,5fc2853c77db91fa3d980f91,https://pantip.com/topic/39868603,นมเมจิรสหวานทำไมหายากจังคะ,ถ้าขายไม่ดีเค้าก็เอาออกครับ shelf life มันน้อย...,2020-05-05 13:22:59,2020-11-29 00:13:26.571


In [14]:
print('All comment:',df_Corpus.shape[0])
print('All URLs ก่อนตัด:',len(df_Corpus.URLs.unique()))

All comment: 63661
All URLs ก่อนตัด: 1516


In [15]:
missing = df_Corpus.isnull().sum()
print(missing)

_id              0
URLs             0
headline         0
text             0
article_date     0
Retrived_date    0
dtype: int64


In [16]:
df_Corpus.tail()

,_id,URLs,headline,text,article_date,Retrived_date
63656,5fc2ec7c77db91fa3d990835,https://pantip.com/topic/30011023,"Fit and Firm to Slim ครั้งที่ 21""New Year New ...",พี่มานะน่าร๊ากมากเลย เห็นแล้วอยากจกพุง,2013-01-04 07:03:18,2020-11-29 07:33:59.076
63657,5fc2ec7c77db91fa3d990836,https://pantip.com/topic/30011023,"Fit and Firm to Slim ครั้งที่ 21""New Year New ...",ลู่ 44 isoulmate เปเป้\n\nเช้า ... แซนวิส\n\nก...,2013-01-04 07:03:18,2020-11-29 07:33:59.076
63658,5fc2ec7c77db91fa3d990837,https://pantip.com/topic/30011023,"Fit and Firm to Slim ครั้งที่ 21""New Year New ...",ลำดับที่ 41\nชื่อlogin โดนัทแสนหวาน \nชื่อเล่น...,2013-01-04 07:03:18,2020-11-29 07:33:59.076
63659,5fc2ec7c77db91fa3d990838,https://pantip.com/topic/30011023,"Fit and Firm to Slim ครั้งที่ 21""New Year New ...",มีใครตั้งทู้ยังเอ่ย\nงั้นเดี๋ยวเค้าตั้งให้ แป้...,2013-01-04 07:03:18,2020-11-29 07:33:59.076
63660,5fc2ec7c77db91fa3d990839,https://pantip.com/topic/30011023,"Fit and Firm to Slim ครั้งที่ 21""New Year New ...",กระทู้วันเสาร์ พร้อมแล้วค่ะ \nhttp://pantip.co...,2013-01-04 07:03:18,2020-11-29 07:33:59.076


In [17]:
df_Corpus.drop('_id',axis=1,inplace=True)

In [18]:
seqNum = list(range(1,df_Corpus.shape[0]+1))
df_Corpus.insert(0,'commentId',seqNum)
df_Corpus.columns = ['commentId','URLs','headline','text','article_date','Retrived_date']

In [19]:
df_Corpus.tail()

,commentId,URLs,headline,text,article_date,Retrived_date
63656,63657,https://pantip.com/topic/30011023,"Fit and Firm to Slim ครั้งที่ 21""New Year New ...",พี่มานะน่าร๊ากมากเลย เห็นแล้วอยากจกพุง,2013-01-04 07:03:18,2020-11-29 07:33:59.076
63657,63658,https://pantip.com/topic/30011023,"Fit and Firm to Slim ครั้งที่ 21""New Year New ...",ลู่ 44 isoulmate เปเป้\n\nเช้า ... แซนวิส\n\nก...,2013-01-04 07:03:18,2020-11-29 07:33:59.076
63658,63659,https://pantip.com/topic/30011023,"Fit and Firm to Slim ครั้งที่ 21""New Year New ...",ลำดับที่ 41\nชื่อlogin โดนัทแสนหวาน \nชื่อเล่น...,2013-01-04 07:03:18,2020-11-29 07:33:59.076
63659,63660,https://pantip.com/topic/30011023,"Fit and Firm to Slim ครั้งที่ 21""New Year New ...",มีใครตั้งทู้ยังเอ่ย\nงั้นเดี๋ยวเค้าตั้งให้ แป้...,2013-01-04 07:03:18,2020-11-29 07:33:59.076
63660,63661,https://pantip.com/topic/30011023,"Fit and Firm to Slim ครั้งที่ 21""New Year New ...",กระทู้วันเสาร์ พร้อมแล้วค่ะ \nhttp://pantip.co...,2013-01-04 07:03:18,2020-11-29 07:33:59.076


In [20]:
# Drop NA & banned comments
df_Corpus = df_Corpus.dropna()
df_Corpus = df_Corpus[df_Corpus["text"].str.find('ความคิดเห็นนี้ถูกลบ')!= 0]
df_Corpus = df_Corpus[df_Corpus["text"].str.find('แก้ไขข้อความเมื่อ')!= 0]
df_Corpus = df_Corpus.set_index('commentId')

In [21]:
missing = df_Corpus.isnull().sum()
print(missing)

URLs             0
headline         0
text             0
article_date     0
Retrived_date    0
dtype: int64


In [22]:
print('จำนวน comment หลังตัด:',df_Corpus.shape[0])
print('จำนวนกระทู้หลังตัด:',len(df_Corpus.URLs.unique()))

จำนวน comment หลังตัด: 62710
จำนวนกระทู้หลังตัด: 1515


In [23]:
df_Corpus.insert(4,'token_text',None)
df_Corpus.tail()

,URLs,headline,text,article_date,token_text,Retrived_date
commentId,,,,,,
63657,https://pantip.com/topic/30011023,"Fit and Firm to Slim ครั้งที่ 21""New Year New ...",พี่มานะน่าร๊ากมากเลย เห็นแล้วอยากจกพุง,2013-01-04 07:03:18,None,2020-11-29 07:33:59.076
63658,https://pantip.com/topic/30011023,"Fit and Firm to Slim ครั้งที่ 21""New Year New ...",ลู่ 44 isoulmate เปเป้\n\nเช้า ... แซนวิส\n\nก...,2013-01-04 07:03:18,None,2020-11-29 07:33:59.076
63659,https://pantip.com/topic/30011023,"Fit and Firm to Slim ครั้งที่ 21""New Year New ...",ลำดับที่ 41\nชื่อlogin โดนัทแสนหวาน \nชื่อเล่น...,2013-01-04 07:03:18,None,2020-11-29 07:33:59.076
63660,https://pantip.com/topic/30011023,"Fit and Firm to Slim ครั้งที่ 21""New Year New ...",มีใครตั้งทู้ยังเอ่ย\nงั้นเดี๋ยวเค้าตั้งให้ แป้...,2013-01-04 07:03:18,None,2020-11-29 07:33:59.076
63661,https://pantip.com/topic/30011023,"Fit and Firm to Slim ครั้งที่ 21""New Year New ...",กระทู้วันเสาร์ พร้อมแล้วค่ะ \nhttp://pantip.co...,2013-01-04 07:03:18,None,2020-11-29 07:33:59.076


เช็ควันเวลา

In [24]:
print('กระทู้เก่าที่สุด :',min(df_Corpus.article_date))
print('กระทู้ใหม่ที่สุด :',max(df_Corpus.article_date))

กระทู้เก่าที่สุด : 2013-01-03 09:20:18
กระทู้ใหม่ที่สุด : 2020-09-11 19:21:41


เก็บเฉพาะหัวกระทู้

In [25]:
df_thread = df_Corpus.copy()
df_thread = df_thread[['URLs','headline']]
df_thread.drop_duplicates(subset ="URLs", keep = 'first', inplace = True)
df_thread.insert(2,'token_headline',None)
df_thread.insert(3,'mention_product',None)
df_thread.insert(4,'mention_brand',None)
df_thread.tail()

,URLs,headline,token_headline,mention_product,mention_brand
commentId,,,,,
63500,https://pantip.com/topic/30059506,อยากทราบว่า ตอนตั้งครรถ์คุณแม่บำรุงอะไรกันบ้าง...,None,None,None
63529,https://pantip.com/topic/30007179,นมยี่ห้ออะไรดีสุดสำหรับคนท้องค่ะ,None,None,None
63551,https://pantip.com/topic/30020409,ฝึกสอนลูกดื่มนม UHT จากกล่องสำเร็จยังไงกันบ้าง...,None,None,None
63578,https://pantip.com/topic/30049123,เมื่อเพื่อนของข้าพเจ้า...เข้าวงการบันเทิง!!! (...,None,None,None
63606,https://pantip.com/topic/30011023,"Fit and Firm to Slim ครั้งที่ 21""New Year New ...",None,None,None


## 2.ตัดคำและเก็บลง MongoDB (ใช้ insight จาก jupyter งานก่อน)

https://www.thainlp.org/pythainlp/tutorials/notebooks/pythainlp-get-started.html#Thai-Characters

In [26]:
garbage_char = ['🥺','\r','\n','\r\n','\xa0','www','application','ทางกฎหมาย','html'
                ,'http','https','url','com','pantip','topic','_','–','พันทิป','^^'
                ,'นะค่ะ','นะคะ','น่ะค่ะ','ฯ','ๆ','แฮ่','เย่ๆๆ','ว๊าว','ฮิ้วววววววววว','Number','กก'
                ,'เมนู','Menu','Net','net','สาขา','บาท','ราคา','ฯ','ๆ','กก','อันนี้','😆', '🤣','😢'
                ,'😏','😂','😿','ววว','xx','อิอิ','แย้ววว','นะคะ','นะค่ะ','มกราคม', 'กุมภาพันธ์'
                ,'มีนาคม', 'เมษายน','พฤษภาคม', 'มิถุนายน', 'กรกฎาคม', 'สิงหาคม','กันยายน', 'ตุลาคม'
                ,'พฤศจิกายน', 'ธันวาคม']
garbage_char  = garbage_char + list_punc

In [27]:
print('จำนวน comment หลังตัด:',df_Corpus.shape[0])
print('จำนวนกระทู้หลังตัด:',len(df_Corpus.URLs.unique()))

จำนวน comment หลังตัด: 62710
จำนวนกระทู้หลังตัด: 1515


In [28]:
custom_list = ['ดัชมิลล์','ดัชมิล','dutch mill','dutchmilk','ดัชชี่','เมจิ','ซีพีเมจิ','ซีพี-เมจิ','cp-meiji','betagen','บีทาเก้น','บีทาเกน'
               ,'โฟร์โมสต์','โฟร์โมส','โฟโมสต์','โฟโมต','โฟรโมสต์','โชคชัย','แดรี่โฮม','dairy home','เดรี่โฮม','อืมม มิลค์'
               ,'Umm! Milk','เอ็มมิลค์','mmilk','เอ็มมิ้ลค์','แมคโนเลีย','นมฟาร์มโชคชัย','นมไทย-เดนมาร์ค','ไทยเดนมาร์ค','วัวแดง','หนองโพ','คาเนชั่น'
               ,'สตอเบอร์รี่','สตรอว์เบอร์รี','รสสตรอเบอรี่','สตรอเบอรี่','สตรอวเบอรี่','สตอเบอรี่','สตรอเบอรี่','รสกาแฟ'
               ,'ชอคโกแล็ต','ช็อกโกแลต','ช็อคโกแลต','ช๊อกโกแลต','ช็อคโกแล็ต','ช็อกโกเลต','มิ้นท์ชอค','มิ้นต์ช้อก','รสกล้วย'
               ,'รสกาแฟ','รสหวาน','รสจืด','ไขมันต่ำ','ไขมัน 0%','low fat','0% fat','ขาดมันเนย','พร่องมันเนย','ไฮแคลเซียม'
               ,'high protein','Hi Protein','HIgh Protein','Whey Formula','วานิลลา'
               ,'ไฮโปรตีน','เวย์โปรตีน','whey','เวย์','อัลมอนด์','ซีเล็ค','ซีเล็คท์','กล้วยหอม','ไวท์คอฟฟี่','คาปูชิโน่','อเมริกาโน่'
               ,'grass fed','free lactose','lactose free','พาสเจอไรซ์','พาสเจอร์ไรซ์','พาสเจอร์ไรส์','พาซเจอไรซ์','พาสเจอร์ไลท์'
               ,'เมจิโกลด์ แม็กซ์','เมจิโกลด์','เมจิโกล์ดแม็กซ์','เมจิ บัลแกเรีย','เมจิบัลแกเรีย','นมเปรี้ยว','รสธรรมชาติ','รสกลมกล่อม'
               ,'โยเกิต','โยเกิรต','โยเกิร์ตพร้อมดื่ม','น้ำตาลมะพร้าว','นมฮอกไกโด','เบดไทม์','ฟรีแลคโตส','แลคโตสฟรี','ริชเอสเพสโซ่'
               ,'bedtime','bed time','เบดไทม์','ดาร์คช็อกโกแลต','ดาร์กช็อก','ดาร์กช็อกโกแลต'
               ,'คาราเมล','ช็อกโกมอลต์','เมล่อน','ชาเขียวมัจฉะ','ฝาน้ำเงิน','ผลิตภันท์นม'
               ,'7-Eleven','เซเว่น อีเลฟเว่น','เซเว่นอีเลฟเว่น','เซเว่น','เซเวน','7/11','7-11'
               ,'ท็อปส์','ทอปส์','ท้อปส์','ท๊อปส์','แมคโคร','แม็คโคร','โลตัส','บิ๊กซี','bigc','Golden place','Big C'
               ,'ขายไม่ดี','แพคคู่','ค่าจัดส่ง','shelf life','พนักงานขายนม'               
               ,'ซื้อประจำ','ซื้อไม่ได้','ของแถม','ราคาสูง','น้ำนมโค'
               ,'แพ้นม','แพ้นมวัว','เล่นเวท','นำ้หนัก','คุณแม่มือใหม่','นมอุ่น','ชานม','กินนม','ดื่มนม','ท้องเสีย','ขี้แตก'
               ,'คีเฟอร์นม','พันทิป','ร้านนม','ซื้อไม่ได้','ลูกสุนัข','ลูกหมา','คายทิ้ง','เจมส์ จิ','เจมส์จิ','สตอรี่'
              ]
words = set(thai_words()).union(set(custom_list))
trie = dict_trie(dict_source=words)

In [29]:
def tokenize_custom(x, cusDict):
    custom_punc = '!"#$&\'()*+,.:;<=>?@[\\]^_`{|}~'
    wtkn = x.translate(str.maketrans('', '', custom_punc)).strip()
    wtkn = word_tokenize(wtkn, engine="newmm", custom_dict=cusDict, keep_whitespace=False)
    wtkn = checkStopwords(wtkn)
    wtkn = [w for w in wtkn if len(w) >= 2]
    wtkn = [y for y in wtkn if y not in garbage_char]
    wtkn = [s for s in wtkn if not s.isdigit()]
    wtkn = normThai(wtkn)
    return wtkn

In [30]:
def create_InsertMany_Thread(df):
    listofdict = []
    for c,lx in enumerate(df.URLs,0):
        info = {
            "URLs": lx,
            "headline": df.headline[c],
            "token_headline":df.token_headline[c],
            "t_mention_dairy":df.t_mention_dairy[c],
            "t_mention_product": df.t_mention_product[c],
            "t_mention_brand": df.t_mention_brand[c]
        }
        listofdict.append(info)
    return listofdict

In [31]:
def create_InsertMany_comments(df):
    listofdict = []
    for c,lx in enumerate(df.URLs,0):
        info = {
            "commentId":int(df.commentId[c]),
            "URLs": lx,
            "headline":df.headline[c],
            "text": df.text[c],
            "Retrived_date": df.Retrived_date[c],
            "token_text":df.token_text[c],
            "cmt_mention_dairy": df.cmt_mention_dairy[c],
            "cmt_mention_product": df.cmt_mention_product[c],
            "cmt_mention_brand": df.cmt_mention_brand[c]
        }
        listofdict.append(info)
    return listofdict

In [32]:
def generalize_keyword(wtkn):  
    ret_wtkn = wtkn
    for i,each in enumerate(ret_wtkn,0):
        if each in ['ดัชมิลล์','ดัชมิล','dutch mill','dutchmill','dutchmilk','ดัชชี่']:
            ret_wtkn[i] = 'ดัชมิลล์'
        elif each  in ['เมจิ','ซีพีเมจิ','ซีพี-เมจิ','cp-meiji','meiji','meji','miji']:
            ret_wtkn[i] = 'เมจิ'
        elif each  in ['โฟร์โมสต์','โฟร์โมส','โฟโมสต์','โฟรโมสต์','โฟโมต','foremost']:
            ret_wtkn[i] = 'โฟร์โมสต์'
        elif each  in ['แดรี่โฮม','dairy home','เดรี่โฮม']:
            ret_wtkn[i] = 'แดรี่โฮม'
        elif each  in ['โชคชัย','อืมม มิลค์','umm milk','นมฟาร์มโชคชัย']:
            ret_wtkn[i] = 'โชคชัย'
        elif each  in ['เอ็มมิลค์','mmilk','เอ็มมิ้ลค์']:
            ret_wtkn[i] = 'เอ็มมิลค์'
        elif each  in ['นมไทย-เดนมาร์ค','ไทยเดนมาร์ค','วัวแดง']:
            ret_wtkn[i] = 'ไทยเดนมาร์ค'
        elif each  in ['สตอเบอร์รี่','สตรอว์เบอร์รี','รสสตรอเบอรี่','สตอเบอรี่','สตรอเบอรี่','สตรอวเบอรี่']:
            ret_wtkn[i] = 'สตรอว์เบอร์รี'
        elif each  in ['ชอคโกแล็ต','ช็อกโกแลต','ช็อคโกแลต','ช๊อกโกแลต','ช็อคโกแล็ต','ช็อกโกเลต','ช็อค']:
            ret_wtkn[i] = 'ช็อกโกแลต'
        elif each  in ['มิ้นท์ชอค','มิ้นต์ช้อก','ช็อคโกแลตมินต์','มินต์ช็อคโกแลต']:
            ret_wtkn[i] = 'ช็อกโกแลตมินต์'
        elif each  in ['ไขมันต่ำ','low fat','พร่องมันเนย']:
            ret_wtkn[i] = 'ไขมันต่ำ'
        elif each  in ['ไขมัน 0','0 fat','ขาดมันเนย']:
            ret_wtkn[i] = 'ไขมัน 0%'
        elif each  in ['high protein','hi protein','ไฮโปรตีน','เวย์โปรตีน']:
            ret_wtkn[i] = 'ไฮโปรตีน'
        elif each  in ['free lactose','lactose free','ฟรีแลคโตส','แลคโตสฟรี']:
            ret_wtkn[i] = 'นมฟรีแลคโตส'
        elif each  in ['พาสเจอไรซ์','พาสเจอร์ไรซ์','พาสเจอร์ไรส์','พาซเจอไรซ์','พาสเจอร์ไลท์']:
            ret_wtkn[i] = 'พาสเจอร์ไรส์'
        elif each  in ['เมจิโกลด์ แม็กซ์','เมจิโกลด์','เมจิโกล์ดแม็กซ์','gold','gold max']:
            ret_wtkn[i] = 'เมจิโกลด์'
        elif each  in ['ฝาน้ำเงิน','รสจืด']:
            ret_wtkn[i] = 'รสจืด'
        elif each  in ['ท็อปส์','ทอปส์','ท้อปส์','ท๊อปส์','tops']:
            ret_wtkn[i] = 'tops'
        elif each  in ['แมคโคร','แม็คโคร','makro']:
            ret_wtkn[i] = 'makro'
        elif each  in ['โลตัส','lotus']:
            ret_wtkn[i] = 'lotus'
        elif each  in ['บิ๊กซี','bigc','Big C']:
            ret_wtkn[i] = 'bigc'
        elif each  in ['ท้องเสีย','ขี้แตก']:
            ret_wtkn[i] = 'ท้องเสีย'
        elif each  in ['ลูกสุนัข','ลูกหมา']:
            ret_wtkn[i] = 'ลูกสุนัข'
        elif each in ['ดาร์คช็อกโกแลต','ดาร์กช็อก','ดาร์กช็อกโกแลต']:
            ret_wtkn[i] = 'ดาร์คช็อกโกแลต'
        elif each in ['bedtime','bed time','เบดไทม์']:
            ret_wtkn[i] = 'เบดไทม์'
        elif each in ['เมจิ บัลแกเรีย','เมจิบัลแกเรีย']:
            ret_wtkn[i] = 'เมจิบัลแกเรีย'
        elif each in ['โยเกิร์ต','โยเกิต','โยเกิรต','โยเกิร์ตพร้อมดื่ม']:
            ret_wtkn[i] = 'โยเกิร์ต'
        elif each in ['7-Eleven','เซเว่น อีเลฟเว่น','เซเว่นอีเลฟเว่น','เซเว่น','เซเวน','7/11','7-11']:
            ret_wtkn[i] = '7-Eleven'
        elif each in ['บีทาเก้น','บีทาเกน']:
            ret_wtkn[i] = 'บีทาเก้น'
        elif each in ['โปร','โปรโมชั่น']:
            ret_wtkn[i] = 'โปรโมชั่น'
    return ret_wtkn

In [33]:
def find_brands_product(x,catType):
    ret_wtkn = []
    brands = ['ดัชมิลล์','เมจิ','โฟร์โมสต์','โชคชัย','แดรี่โฮม','เอ็มมิลค์','แมคโนเลีย','ไทยเดนมาร์ค','หนองโพ','คาเนชั่น','บีทาเก้น','จิตรลดา']
    products = ['สตรอว์เบอร์รี','ช็อกโกแลต','รสกาแฟ','รสหวาน','รสจืด','ไขมันต่ำ','ไขมัน 0%','ไฮโปรตีน','อัลมอนด์'
                ,'รสกล้วย','grass fed','นมฟรีแลคโตส','เมจิโกลด์','นมฮอกไกโด','เบดไทม์','ดาร์คช็อกโกแลต','ไฮแคลเซียม'
                ,'คาราเมล','มอลต์','เมล่อน','ชาเขียวมัจฉะ','บัลแกเรีย','รสธรรมชาติ','รสกลมกล่อม','ซากุระ']
    milk_kind=['นม','นมข้น','นมจืด','นมสด','กินนม','ดื่มนม','ขวดนม','นมวัว','นมกล่อง','ผลิตภันท์นม','น้ำนมโค'
                     ,'โยเกิร์ต','นมเปรี้ยว','uht','นมถั่วเหลือง','นมผง','พาสเจอร์ไรส์']
    
    choiceList = []
    if catType == 'b':
        choiceList = brands
    elif catType == 'm':
        choiceList = milk_kind
    elif catType == 'p':
        choiceList = products
        
    for i,each in enumerate(x,0):
        if each in choiceList:
            ret_wtkn.append(each)
    return ret_wtkn

#### Copy ชุดคำจาก File มา Process ก่อนลง MongoDB

In [34]:
df_process_comment = df_Corpus.reset_index().copy()
df_process_thread = df_thread.reset_index().copy()

2.1) Tokenize comment

In [35]:
df_process_comment['token_text'] = df_process_comment['text'].apply(lambda x: tokenize_custom(x,trie))

In [36]:
df_process_comment.head()

,commentId,URLs,headline,text,article_date,token_text,Retrived_date
0,1,https://pantip.com/topic/39868603,นมเมจิรสหวานทำไมหายากจังคะ,เมื่อก่อนหลายปีมาแล้ว ในเซเว่นยังมีนมเมจิรสหวา...,2020-05-05 13:22:59,"[ปี, เซเว่น, ยังมี, นม, เมจิ, รสหวาน, ขาย, อยู...",2020-11-29 00:13:26.571
1,2,https://pantip.com/topic/39868603,นมเมจิรสหวานทำไมหายากจังคะ,เบาหวานครับ แฮ่ๆ,2020-05-05 13:22:59,[เบาหวาน],2020-11-29 00:13:26.571
2,3,https://pantip.com/topic/39868603,นมเมจิรสหวานทำไมหายากจังคะ,หาซื้อไม่ได้นี่คิดได้ 2 อย่างนะ. คนไม่ค่อย...,2020-05-05 13:22:59,"[หา, ซื้อไม่ได้, คิดได้, , คน, กินกัน, ไม่, ทำ...",2020-11-29 00:13:26.571
3,4,https://pantip.com/topic/39868603,นมเมจิรสหวานทำไมหายากจังคะ,เป็นรสที่ขายไม่ดีครับ\nรสพื้นฐานคือ\nจืด แบ่งย...,2020-05-05 13:22:59,"[รส, ขายไม่ดี, รส, พื้นฐาน, จืด, แบ่ง, low fat...",2020-11-29 00:13:26.571
4,5,https://pantip.com/topic/39868603,นมเมจิรสหวานทำไมหายากจังคะ,ถ้าขายไม่ดีเค้าก็เอาออกครับ shelf life มันน้อย...,2020-05-05 13:22:59,"[ขายไม่ดี, เค้า, shelf life, เวลา, หมดอายุ, ค่...",2020-11-29 00:13:26.571


2.2) Generalized keyword (Brand & product)

In [37]:
df_process_comment['token_text'] = df_process_comment['token_text'].apply(lambda x: generalize_keyword(x))

In [38]:
df_process_comment['cmt_mention_dairy'] = df_process_comment['token_text'].apply(lambda x: find_brands_product(x,'m'))
df_process_comment['cmt_mention_brand'] = df_process_comment['token_text'].apply(lambda x: find_brands_product(x,'b'))
df_process_comment['cmt_mention_product'] = df_process_comment['token_text'].apply(lambda x: find_brands_product(x,'p'))

2.3) Insert comments to DB

In [39]:
col_comment.delete_many({})
process_data = create_InsertMany_comments(df_process_comment)
col_comment.insert_many(process_data)

2.4) Tokenize head of thread

In [40]:
df_process_thread['token_headline'] = df_process_thread['headline'].apply(lambda x: tokenize_custom(x,trie))

2.5) Generalized keyword (Brand & product)

In [41]:
df_process_thread['token_headline'] = df_process_thread['token_headline'].apply(lambda x: generalize_keyword(x))

2.6) Mark domain/entity mention

In [42]:
df_process_thread['t_mention_dairy'] = df_process_thread['token_headline'].apply(lambda x: find_brands_product(x,'m'))
df_process_thread['t_mention_brand'] = df_process_thread['token_headline'].apply(lambda x: find_brands_product(x,'b'))
df_process_thread['t_mention_product'] = df_process_thread['token_headline'].apply(lambda x: find_brands_product(x,'p'))

In [43]:
df_process_thread.head()

,commentId,URLs,headline,token_headline,mention_product,mention_brand,t_mention_dairy,t_mention_brand,t_mention_product
0,1,https://pantip.com/topic/39868603,นมเมจิรสหวานทำไมหายากจังคะ,"[นม, เมจิ, รสหวาน, หา]",None,None,[นม],[เมจิ],[รสหวาน]
1,8,https://pantip.com/topic/39597709,ระหว่างนมเมจิต้มกับนมเมจิที่ไม่ได้ต้ม เมื่อนำไ...,"[นม, เมจิ, ต้ม, นม, เมจิ, ไม่, ได้, ต้ม, ทำ, เ...",None,None,"[นม, นม]","[เมจิ, เมจิ]",[]
2,14,https://pantip.com/topic/39747201,ทำไมนมเมจิขวดใหญ่ 2 ลิตร ไม่มีซีลที่ปากแบบขวดเล็ก,"[นม, เมจิ, ขวด, ลิตร, ไม่, ซีล, ปาก, ขวด]",None,None,[นม],[เมจิ],[]
3,15,https://pantip.com/topic/39732615,Makro มีนม meiji(เมจิ) ขายมั้ยครัย,"[makro, นม, เมจิ, เมจิ, ขาย, รัย]",None,None,[นม],"[เมจิ, เมจิ]",[]
4,20,https://pantip.com/topic/39005469,ใครเคยสั่งซื้อนมเมจิ (Meiji) ให้มาส่งครั้งละเย...,"[สั่งซื้อ, นม, เมจิ, เมจิ]",None,None,[นม],"[เมจิ, เมจิ]",[]


2.7) Insert thread to DB

In [44]:
col_thread.delete_many({})
process_data = create_InsertMany_Thread(df_process_thread)
col_thread.insert_many(process_data)

In [45]:
del df_thread,df_Corpus,df_process_comment,df_process_thread

### 3.Process หัวกระทู้ เพื่อทำ Topic Modeling

เคสหลุดเงื่อนไข แต่ต้องใช้
1. https://pantip.com/topic/30833944
2. https://pantip.com/topic/30105850
4. https://pantip.com/topic/35439062

In [46]:
execption_urls = ['https://pantip.com/topic/30833944','https://pantip.com/topic/30105850','https://pantip.com/topic/35439062']

In [47]:
cursor_thread = col_thread.find()
df_thr_process = pd.DataFrame(cursor_thread)

In [48]:
cursor_comment = col_comment.find()
df_cmt_process = pd.DataFrame(cursor_comment)

In [49]:
df_urls_milk = df_thr_process[(df_thr_process.t_mention_dairy.str.len() != 0) | (df_thr_process.URLs.isin(execption_urls))]
df_urls_milk['t_mention_dairy'] = df_urls_milk['t_mention_dairy'].apply(lambda x: repr(set(x)))
df_urls_milk['t_mention_brand'] = df_urls_milk['t_mention_brand'].apply(lambda x: repr(set(x)))
df_urls_milk['t_mention_product'] = df_urls_milk['t_mention_product'].apply(lambda x: repr(set(x)))

In [50]:
df_c_urls_milk = df_cmt_process[(df_cmt_process.cmt_mention_dairy.str.len() != 0) | (df_cmt_process.URLs.isin(execption_urls))]
df_c_urls_milk['cmt_mention_dairy'] = df_c_urls_milk['cmt_mention_dairy'].apply(lambda x: repr(set(x)))
df_c_urls_milk['cmt_mention_brand'] = df_c_urls_milk['cmt_mention_brand'].apply(lambda x: repr(set(x)))
df_c_urls_milk['cmt_mention_product'] = df_c_urls_milk['cmt_mention_product'].apply(lambda x: repr(set(x)))

In [51]:
print('จำนวนกระทู้ที่มีการพูดถึงนม:',df_urls_milk.shape[0])
print('%จำนวนกระทู้ที่มีการพูดถึงนมเทียบทั้งหมด:',round((df_urls_milk.shape[0]/df_thr_process.shape[0])*100,2))
print('จำนวน comment ที่มีการพูดถึงนม:',df_c_urls_milk.shape[0])
print('%จำนวน comment ที่มีการพูดถึงนมเทียบทั้งหมด:',round((df_c_urls_milk.shape[0]/df_cmt_process.shape[0])*100,2))

จำนวนกระทู้ที่มีการพูดถึงนม: 442
%จำนวนกระทู้ที่มีการพูดถึงนมเทียบทั้งหมด: 29.17
จำนวน comment ที่มีการพูดถึงนม: 8542
%จำนวน comment ที่มีการพูดถึงนมเทียบทั้งหมด: 13.62


* จำนวน comment ที่มีการพูดถึงนม ไม่ได้ impute มาจากหัวกระทู้ ถ้า impute จะมีเยอะกว่านี้อีก

### EDA#2

การระบุ domain ของนม บนหัวกระทู้

In [52]:
df_view = df_urls_milk[['URLs','t_mention_dairy']]
df_view.column = ['URLs','t_mention_dairy']
df_view_count = df_view.groupby('t_mention_dairy').count()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  


In [53]:
df_view_count.sort_values(by='URLs',ascending=False)[0:20]

# set() ในเคสนี้คือกระทู้ที่พูดถึงนมแต่ไม่มี domain ในหัวกระทู้ ต้องดึง custom

,URLs
t_mention_dairy,
{'นม'},216
{'นมสด'},41
{'กินนม'},39
{'ดื่มนม'},31
{'โยเกิร์ต'},28
{'นมวัว'},8
"{'พาสเจอร์ไรส์', 'นม'}",7
"{'กินนม', 'นม'}",7
"{'นมสด', 'นม'}",4


การระบุชื่อ Product บนหัวกระทู้ (Headline)

In [54]:
df_view = df_urls_milk[['URLs','t_mention_product']]
df_view.column = ['URLs','t_mention_product']
df_view_count = df_view.groupby('t_mention_product').count()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  


In [55]:
df_view_count.sort_values(by='URLs',ascending=False)[0:20]

# set() หมายถึง กระทู้นั้นอยู่ใน domain นม แต่ไม่ระบุ product flavor

,URLs
t_mention_product,
set(),389
{'รสหวาน'},10
{'ไฮโปรตีน'},10
{'รสจืด'},8
{'ไขมัน 0%'},5
{'ไขมันต่ำ'},5
{'ช็อกโกแลต'},3
{'นมฮอกไกโด'},2
{'รสกาแฟ'},2


การระบุชื่อ Product ใน comment ทั้งใต้หัวกระทู้ ที่มี domain ของนม และส่วน comment<br>
อย่างไรก็ตามยังไม่ได้ impute domain จากหัวกระทู้เข้าไป

In [56]:
df_view = df_c_urls_milk[['URLs','cmt_mention_product']]
df_view.column = ['URLs','cmt_mention_product']
df_view_count = df_view.groupby('cmt_mention_product').count()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  


In [57]:
df_view_count.sort_values(by='URLs',ascending=False)

,URLs
cmt_mention_product,
set(),6939
{'รสจืด'},237
{'ไขมันต่ำ'},160
{'ช็อกโกแลต'},152
{'รสหวาน'},89
...,...
"{'รสจืด', 'รสหวาน', 'ช็อกโกแลต', 'รสกล้วย', 'เมล่อน', 'สตรอว์เบอร์รี'}",1
"{'รสจืด', 'รสธรรมชาติ', 'บัลแกเรีย'}",1
"{'รสจืด', 'รสกล้วย'}",1


การระบุชื่อ brand ใน comment

In [58]:
df_view = df_c_urls_milk[['URLs','cmt_mention_brand']]
df_view.column = ['URLs','cmt_mention_brand']
df_view_count = df_view.groupby('cmt_mention_brand').count()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  


In [59]:
df_view_count.sort_values(by='URLs',ascending=False)[0:20]

,URLs
cmt_mention_brand,
set(),5879
{'เมจิ'},1160
{'โฟร์โมสต์'},315
{'ดัชมิลล์'},238
{'โชคชัย'},122
{'ไทยเดนมาร์ค'},98
{'จิตรลดา'},73
{'หนองโพ'},69
{'แดรี่โฮม'},46


### 4.ทำ Topic Modeling และคัดเลือกกระทู้ที่มีความเกี่ยวข้อง แล้วดึงคำไปใช้

### 5.get_term_topics เทียบคำที่ได้ไป Review Literature ตามค่าความน่าจะเป็น แล้วสรุป Term

https://radimrehurek.com/gensim/models/ldamodel.html

<b>Domain ที่พบหลักๆ นอกเหนือจากตัว Product ห้างที่ขาย แบรนด์ (อย่างไรก็ตามต้องทำ LDA Vis อีกรอบเพื่อจัดกลุ่มใหม่หลังคัดเลือก Topic/Term)</b><br>
ค้นหาเทียบเคียงกับ ปัจจัยที่มีผลต่อการบริโภคนม
1. แม่และเด็ก (Kurajdova et al, 2015)
2. ออกกำลังกาย เล่นเวท วิ่ง พูดถึง Whey หางนม โปรตีน เป็นหลัก (คิดว่าแตกจากหมวดสุขภาพ)
3. โปรโมชั่น เช่น จัดโปร ส่วนลด พันแถม แพ๊กคู่ (สุภชาติ ชัยณรงค์สิงห์, 2539)
4. เอานมให้หมาแมวกิน (เทรนด์ใหม่)
5. รสชาติและคุณภาพ เช่น หอม เหม็น มัน อร่อย บูด เสีย GMP (ชาลิสา สถีระกานนท์, 2560)
6. ความยากง่ายในการหาซื้อ (Aen De Alwis et al, 2009 & สุภชาติ ชัยณรงค์สิงห์, 2539)
7. ทำกาแฟ ทำขนม (Kurajdova et al, 2015)
8. สุขภาพ ความสูง ส่วนผสมเพื่อสุขภาพ การขับถ่าย อนุมูลอิสระ (Kurajdova et al, 2015 & Aen De Alwis et al, 2009 & สุภชาติ ชัยณรงค์สิงห์, 2539)
9. ราคาและความคุ้มค่า (ปิยฉัตร ช่างเหล็ก, 2561)
<br><br>ตรงนี้ต้องหา Paper มาช่วยยันในบางหัวข้อ ถ้ามีส่วนเกินถือว่าเป็นบริบทของประเทศไทย

### 6.สรุป Term & reduced_keyword

In [60]:
execption_urls = ['https://pantip.com/topic/30833944','https://pantip.com/topic/30105850','https://pantip.com/topic/35439062']

In [61]:
attrributes = ['การบูด','โปร','โปรโมชั่น','แถม','บูด','เป็นผลดี','หา','หมดอายุ', 'กลิ่น','หอม','กลิ่นหอม', 'แนะนำ', 'กิน', 'shelf life','แม่','คุณแม่'
 ,'ลูก','เด็ก','เลี้ยงลูก','เลี้ยงเด็ก','พัฒนาการ','คุณแม่มือใหม่','สั่ง', 'ค่าใช้จ่าย', 'เพิ่มขึ้น', 'สินค้า', 'ชอบ','นิยม', 'สี', 'ซื้อไม่ได้'
 , 'ทิ้ง', 'จืด','หวาน' , 'fat', 'นิยม','ราคาสูง','กาแฟ','ชงกาแฟ','หาย','ต้ม','อุ่น','ร้อน', 'พื้นฐาน', 'ขายไม่ดี','สูง','ส่วนสูง'
 , 'กาแฟสด', 'จ่าย', 'ขาย', 'รส','รสชาติ', 'กลมกล่อม','ไขมัน','พนักงาน','ไมโครเวฟ','ฟาร์ม'
 ,'ส่วนผสม','ผสม','เบาหวาน','รสพื้นฐาน','โปรตีน','เวย์','เวท','วิ่ง','protein','whey','ขับถ่าย','อึ','dha','น้ำตาล','เบาหวาน'
 ,'ฝา', 'นมสด','ถ้วย', 'ร้าน', 'ดื่ม', 'เซเว่น', 'ขวด', 'นึกถึง', 'เสียใจ', 'แตก', 'ซื้อ','ขาย','มีลูก','เทรนเนอร์'
 ,'ขายไม่ดี','แพคคู่','ค่าจัดส่ง','shelf life','พนักงานขายนม','แพ้นม','แพ้นมวัว','เล่นเวท','ฝาน้ำเงิน','ฝาสีเขียว'
 ,'นมอุ่น','ชานม','กินนม','ดื่มนม','ท้องเสีย','ลูกสุนัข','สุนัข','หมา','แมว','คายทิ้ง','เจมส์จิ','แป้ง','ลดราคา'
 ,'ซื้อประจำ','ซื้อไม่ได้','คาปูชิโน่','อเมริกาโน่','ร้านนม','whey formula','ผิดสังเกต','เสียความรู้สึก','ชี้แจง','บำรุง','น้ำผึ้ง'
 ,'อาหารเสริม','มีประโยชน์','วิตามิน','นมผง','แคลเซี่ยม','ท้อง','แพะ','อร่อย','โภชนาการ','โรงเรียน','พ่อแม่','ครู','ยูเอชที'
 ,'ผู้บริโภค','ขโมย','ไอโอดีน','นมข้นหวาน','เนย','ตรวจสอบ','แกลลอน','มันดี','นมวัว','อาเจียน','เวฟ','ไมโครเวฟ','สารอาหาร'
 ,'ขนส่ง','ถูก','แพง','เค้ก','ทิ้ง','วันหมดอายุ','โอเมก้า','กล่อง','พลังงาน','โภชนาการ','ขนมปัง','ของแถม','ราคาสูง','น้ำนมโค'
 ,'บรรจุภัณฑ์','นมถั่วเหลือง','ความแข็งแรง','แข็งแรง','พรีเซ็นเตอร์','ญี่ปุ่น','น้ำผลไม้','ออกกำลังกาย','ประหยัด','วิปปิ้ง','เชื้อจุลินทรีย์'
 ,'แลคโตส']

stores = ['tops','makro','lotus','bigc','7-Eleven']

reduceCol = attrributes + stores

In [62]:
len(reduceCol)

176

In [63]:
def reduced_keyword(wtkn,redCol):
    del_list = []
    ret_wtkn = wtkn
    for each in ret_wtkn:
        if each not in redCol:
            del_list.append(each)
    ret_wtkn = [x for x in ret_wtkn if x not in del_list]
    return ret_wtkn

### 7.สร้าง (Custom) Bag of word ด้วย dictionary.doc2bow จัดลง dataframe

In [64]:
cursor_thread = col_thread.find()
df_thr_process = pd.DataFrame(cursor_thread)

In [65]:
cursor_comment = col_comment.find()
df_cmt_process = pd.DataFrame(cursor_comment)

In [66]:
# คัด headline ที่พูดถึง domain นม
df_urls_milk = df_thr_process[(df_thr_process.t_mention_dairy.str.len() != 0) | (df_thr_process.URLs.isin(execption_urls))]
df_urls_milk['t_mention_dairy'] = df_urls_milk['t_mention_dairy'].apply(lambda x: repr(x))
df_urls_milk['t_mention_brand'] = df_urls_milk['t_mention_brand'].apply(lambda x: repr(x))
df_urls_milk['t_mention_product'] = df_urls_milk['t_mention_product'].apply(lambda x: repr(x))

df_cmt_process['cmt_mention_dairy'] = df_cmt_process['cmt_mention_dairy'].apply(lambda x: repr(x))
df_cmt_process['cmt_mention_brand'] = df_cmt_process['cmt_mention_brand'].apply(lambda x: repr(x))
df_cmt_process['cmt_mention_product'] = df_cmt_process['cmt_mention_product'].apply(lambda x: repr(x))

In [67]:
# เมื่อ join กันแล้ว คอมเม้นทุกคอมเม้นจะอยู่ใน domain นมทั้งหมด
df_join_url = pd.merge(df_urls_milk,df_cmt_process,how='inner',on='URLs')

In [68]:
df_join_url.drop(columns=['_id_x','_id_y','headline_y'],inplace=True)
df_join_url.rename(columns={"headline_x":"headline"},inplace=True)

In [69]:
df_join_url['token_text_reduce'] = df_join_url['token_text'].apply(lambda x: reduced_keyword(x,reduceCol))

In [70]:
df_join_url.set_index('commentId',inplace=True)
df_join_url.head(2)

,URLs,headline,token_headline,t_mention_dairy,t_mention_product,t_mention_brand,text,Retrived_date,token_text,cmt_mention_dairy,cmt_mention_product,cmt_mention_brand,token_text_reduce
commentId,,,,,,,,,,,,,
1,https://pantip.com/topic/39868603,นมเมจิรสหวานทำไมหายากจังคะ,"[นม, เมจิ, รสหวาน, หา]",['นม'],['รสหวาน'],['เมจิ'],เมื่อก่อนหลายปีมาแล้ว ในเซเว่นยังมีนมเมจิรสหวา...,2020-11-29 00:13:26.571,"[ปี, 7-Eleven, ยังมี, นม, เมจิ, รสหวาน, ขาย, อ...","['นม', 'นม']","['รสหวาน', 'รสหวาน']",['เมจิ'],"[7-Eleven, ขาย, หาย, หา, tops, ขวด, ขวด, หา, ร..."
2,https://pantip.com/topic/39868603,นมเมจิรสหวานทำไมหายากจังคะ,"[นม, เมจิ, รสหวาน, หา]",['นม'],['รสหวาน'],['เมจิ'],เบาหวานครับ แฮ่ๆ,2020-11-29 00:13:26.571,[เบาหวาน],[],[],[],[เบาหวาน]


In [71]:
dictionary = gensim.corpora.Dictionary(df_join_url['token_text_reduce'])

In [72]:
df_join_url['reduce_bow'] = df_join_url["token_text_reduce"].map(dictionary.doc2bow)
df_join_url['reduce_bow_txt'] = df_join_url["reduce_bow"].apply(lambda x:[(dictionary[id_], frequence) for id_, frequence in x])

In [73]:
def freq_brand(x):
    e = eval(x)
    e.sort()
    f = [(k,len(list(g))) for k, g in groupby(e)]
    return f

In [74]:
df_join_url['t_mention_dairy'] = df_join_url['t_mention_dairy'].apply(lambda x: freq_brand(x))
df_join_url['t_mention_product'] = df_join_url['t_mention_product'].apply(lambda x: freq_brand(x))
df_join_url['t_mention_brand'] = df_join_url['t_mention_brand'].apply(lambda x: freq_brand(x))
df_join_url['cmt_mention_dairy'] = df_join_url['cmt_mention_dairy'].apply(lambda x: freq_brand(x))
df_join_url['cmt_mention_product'] = df_join_url['cmt_mention_product'].apply(lambda x: freq_brand(x))
df_join_url['cmt_mention_brand'] = df_join_url['cmt_mention_brand'].apply(lambda x: freq_brand(x))
df_join_url.drop(columns=['Retrived_date','reduce_bow'],inplace=True)

In [75]:
df_join_url.insert(3,'mention_domain',value=None)
df_join_url.insert(4,'mention_product',value=None)
df_join_url.insert(5,'mention_brand',value=None)

Impute brand & product เข้า comment และสรุปก่อนเข้า BoW

In [76]:
for i,e in enumerate(df_join_url.index,0):
    if (len(df_join_url.t_mention_dairy.iloc[i]) > 0) & (len(df_join_url.cmt_mention_dairy.iloc[i]) > 0):         # 1.
        df_join_url['mention_domain'].iloc[i] = df_join_url.cmt_mention_dairy.iloc[i]
    elif  (len(df_join_url.t_mention_dairy.iloc[i]) > 0) & (len(df_join_url.cmt_mention_dairy.iloc[i]) == 0):  # 2.
        df_join_url['mention_domain'].iloc[i] = df_join_url.t_mention_dairy.iloc[i]
    elif  (len(df_join_url.t_mention_dairy.iloc[i]) == 0) & (len(df_join_url.cmt_mention_dairy.iloc[i]) > 0):  # 3.
        df_join_url['mention_domain'].iloc[i] = df_join_url.cmt_mention_dairy.iloc[i]
    else:
        df_join_url['mention_domain'].iloc[i] = None

In [77]:
for i,e in enumerate(df_join_url.index,0):
    if (len(df_join_url.t_mention_brand.iloc[i]) > 0) & (len(df_join_url.cmt_mention_brand.iloc[i]) > 0):         # 1.
        df_join_url['mention_brand'].iloc[i] = df_join_url.cmt_mention_brand.iloc[i]
    elif  (len(df_join_url.t_mention_brand.iloc[i]) > 0) & (len(df_join_url.cmt_mention_brand.iloc[i]) == 0):  # 2.
        df_join_url['mention_brand'].iloc[i] = df_join_url.t_mention_brand.iloc[i]
    elif  (len(df_join_url.t_mention_brand.iloc[i]) == 0) & (len(df_join_url.cmt_mention_brand.iloc[i]) > 0):  # 3.
        df_join_url['mention_brand'].iloc[i] = df_join_url.cmt_mention_brand.iloc[i]
    else:
        df_join_url['mention_brand'].iloc[i] = None

In [78]:
for i,e in enumerate(df_join_url.index,0):
    if (len(df_join_url.t_mention_product.iloc[i]) > 0) & (len(df_join_url.cmt_mention_product.iloc[i]) > 0):         # 1.
        df_join_url['mention_product'].iloc[i] = df_join_url.cmt_mention_product.iloc[i]
    elif  (len(df_join_url.t_mention_product.iloc[i]) > 0) & (len(df_join_url.cmt_mention_product.iloc[i]) == 0):  # 2.
        df_join_url['mention_product'].iloc[i] = df_join_url.t_mention_product.iloc[i]
    elif  (len(df_join_url.t_mention_product.iloc[i]) == 0) & (len(df_join_url.cmt_mention_product.iloc[i]) > 0):  # 3.
        df_join_url['mention_product'].iloc[i] = df_join_url.cmt_mention_product.iloc[i]
    else:
        df_join_url['mention_product'].iloc[i] = None

In [79]:
df_join_url.drop(columns=['t_mention_dairy','t_mention_product','t_mention_brand'
                          ,'cmt_mention_dairy','cmt_mention_product','cmt_mention_brand'],inplace=True)

In [80]:
df_join_url.tail(30)

,URLs,headline,token_headline,mention_domain,mention_product,mention_brand,text,token_text,token_text_reduce,reduce_bow_txt
commentId,,,,,,,,,,
63548,https://pantip.com/topic/30007179,นมยี่ห้ออะไรดีสุดสำหรับคนท้องค่ะ,"[นม, ยี่ห้อ, ดี, สำหรับ, คน, ท้อง]","[(นม, 1)]",None,None,เราว่าไม่จำเป็นต้องบำรุงอะไรมากนะ แค่ทานอาหารใ...,"[ไม่, จำเป็นต้อง, บำรุง, ทานอาหาร, หมู่, เผลอ,...","[บำรุง, บำรุง]","[(บำรุง, 2)]"
63549,https://pantip.com/topic/30007179,นมยี่ห้ออะไรดีสุดสำหรับคนท้องค่ะ,"[นม, ยี่ห้อ, ดี, สำหรับ, คน, ท้อง]","[(นมถั่วเหลือง, 1), (นมวัว, 1)]",None,None,เราไม่ดื่มนมวัวเลยเพราะแพ้ ดื่มแล้วท้องเสีย ช่...,"[ไม่, ดื่ม, นมวัว, แพ้, ดื่ม, ท้องเสีย, ท้อง, ...","[ดื่ม, นมวัว, ดื่ม, ท้องเสีย, ท้อง, ดื่ม, นมถั...","[(กิน, 4), (ดื่ม, 4), (ลูก, 1), (วิตามิน, 1), ..."
63550,https://pantip.com/topic/30007179,นมยี่ห้ออะไรดีสุดสำหรับคนท้องค่ะ,"[นม, ยี่ห้อ, ดี, สำหรับ, คน, ท้อง]","[(นม, 2), (นมถั่วเหลือง, 1), (นมสด, 1), (นมเปร...",None,None,เราทานนมทุกชนิดเลยค่ะ นมสด นมถั่วเหลือง นมเปรี...,"[ทาน, นม, ทุกชนิด, นมสด, นมถั่วเหลือง, นมเปรี้...","[นมสด, นมถั่วเหลือง, กิน, อร่อย]","[(กิน, 1), (นมสด, 1), (อร่อย, 1), (นมถั่วเหลือ..."
63551,https://pantip.com/topic/30020409,ฝึกสอนลูกดื่มนม UHT จากกล่องสำเร็จยังไงกันบ้าง...,"[ฝึกสอน, ลูก, ดื่มนม, uht, กล่อง, สำเร็จ, แชร์...","[(นม, 1)]",None,None,พรุ่งนี้ลูกชายอายุครบ 1 ขวบ 3 เดือน หม่ำนมแม่จ...,"[พรุ่งนี้, ลูกชาย, อายุ, ขวบ, เดือน, หม่ำ, นม,...","[แม่, ขวด, กล่อง, คุณแม่, ลูก]","[(ขวด, 1), (กล่อง, 1), (ลูก, 1), (แม่, 1), (คุ..."
63552,https://pantip.com/topic/30020409,ฝึกสอนลูกดื่มนม UHT จากกล่องสำเร็จยังไงกันบ้าง...,"[ฝึกสอน, ลูก, ดื่มนม, uht, กล่อง, สำเร็จ, แชร์...","[(กินนม, 1), (นม, 2), (พาสเจอร์ไรส์, 1)]",None,None,ของเราให้นมพาสเจอร์ไรส์ (อยู่ ตปท.) ตอนแรกเราผ...,"[ให้นม, พาสเจอร์ไรส์, ตปท, ตอนแรก, ผสม, นม, แม...","[ผสม, แม่, ถ้วย, ลูก, ดื่ม, กิน, ผสม, กิน, ดื่...","[(กิน, 3), (ดื่ม, 2), (กินนม, 1), (ผสม, 2), (ล..."
63553,https://pantip.com/topic/30020409,ฝึกสอนลูกดื่มนม UHT จากกล่องสำเร็จยังไงกันบ้าง...,"[ฝึกสอน, ลูก, ดื่มนม, uht, กล่อง, สำเร็จ, แชร์...","[(นม, 1)]",None,None,ลองให้เค้าดูดจากกล่องแล้วเค้าไม่เอาเลยค่ะ แต่น...,"[ลอง, เค้า, ดูด, กล่อง, เค้า, ไม่, เอาเลย, น้ำ...","[กล่อง, น้ำผลไม้, ชอบ, ดื่ม, กล่อง, ผสม, แม่]","[(ดื่ม, 1), (ชอบ, 1), (กล่อง, 2), (ผสม, 1), (แ..."
63554,https://pantip.com/topic/30020409,ฝึกสอนลูกดื่มนม UHT จากกล่องสำเร็จยังไงกันบ้าง...,"[ฝึกสอน, ลูก, ดื่มนม, uht, กล่อง, สำเร็จ, แชร์...","[(uht, 1), (กินนม, 1), (นมถั่วเหลือง, 1), (นมเ...",None,None,เอ่อ..ของผม 1.1 ขวบครับดูดเองได้แล้ว แต่ให้เค้...,"[เอ่อ, ผม, ขวบ, ดูด, ได้, เค้า, ได้, ชอบ, บีบ,...","[ชอบ, กล่อง, ลูก, กล่อง, กล่อง, กินนม, กิน, น้...","[(กิน, 2), (ชอบ, 1), (กินนม, 1), (กล่อง, 3), (..."
63555,https://pantip.com/topic/30020409,ฝึกสอนลูกดื่มนม UHT จากกล่องสำเร็จยังไงกันบ้าง...,"[ฝึกสอน, ลูก, ดื่มนม, uht, กล่อง, สำเร็จ, แชร์...","[(นม, 1), (นมเปรี้ยว, 1)]",None,None,ลูกชายยยยชอบดูด น้ำผลไม้ นมเปรี้ยว น้ำ ยกเว้นน...,"[ลูกชาย, ยยย, ชอบ, ดูด, น้ำผลไม้, นมเปรี้ยว, น...","[ชอบ, น้ำผลไม้]","[(ชอบ, 1), (น้ำผลไม้, 1)]"
63556,https://pantip.com/topic/30020409,ฝึกสอนลูกดื่มนม UHT จากกล่องสำเร็จยังไงกันบ้าง...,"[ฝึกสอน, ลูก, ดื่มนม, uht, กล่อง, สำเร็จ, แชร์...","[(นมกล่อง, 2)]",None,None,เหมือนกันเลยค่ะ คุณแม่ ดูดกินอย่างอื่นได้หมดทุ...,"[เหมือนกัน, คุณแม่, ดูด, กิน, ได้, ยกเว้น, นมก...","[คุณแม่, กิน, ขวด, กิน]","[(ขวด, 1), (กิน, 2), (คุณแม่, 1)]"


Create cooccurence matrix

In [81]:
df_brand_cooc = pd.concat([pd.DataFrame(s,columns=['w','c']).set_index('w') for s in df_join_url["mention_brand"]], axis=1, sort=False).fillna(0).T.set_index(df_join_url.index)
df_prd_cooc = pd.concat([pd.DataFrame(s,columns=['w','c']).set_index('w') for s in df_join_url["mention_product"]], axis=1, sort=False).fillna(0).T.set_index(df_join_url.index)
df_cooc = pd.concat([pd.DataFrame(s,columns=['w','c']).set_index('w') for s in df_join_url["reduce_bow_txt"]], axis=1, sort=False).fillna(0).T.set_index(df_join_url.index)

In [82]:
df_brand_cooc.reset_index(inplace=True)
df_prd_cooc.reset_index(inplace=True)
df_cooc.reset_index(inplace=True)

In [83]:
join_df = pd.merge(df_brand_cooc, df_prd_cooc, on='commentId', how='inner', suffixes=(False, False))
join_df = pd.merge(join_df, df_cooc, on='commentId', how='inner')
join_df.set_index('commentId',inplace=True)

In [84]:
join_df.head()

,เมจิ,ไทยเดนมาร์ค,ดัชมิลล์,แมคโนเลีย,จิตรลดา,แดรี่โฮม,โฟร์โมสต์,โชคชัย,คาเนชั่น,หนองโพ,...,พนักงานขายนม,ความแข็งแรง,ผิดสังเกต,เสียความรู้สึก,ไอโอดีน,พรีเซ็นเตอร์,สุนัข,คายทิ้ง,มีลูก,คุณแม่มือใหม่
commentId,,,,,,,,,,,,,,,,,,,,,
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Create frequency co-occurrence matrix จาก Bag of word<br>
* ใช้สำหรับทำ MDS เท่านั้น

In [85]:
item_item_matrix = pd.DataFrame(index=join_df.columns,columns=join_df.columns).fillna(0)

In [86]:
def cnt_Grpby(df,i,j):
    PAIB = pd.DataFrame(df.groupby(df.iloc[:,i].name)[df.iloc[:,j].name].value_counts())
    PAIB.index.names = ['A','B']
    PAIB.columns = ['freq']
    PAIB.reset_index(inplace=True)
    _PAIB = PAIB[(PAIB.A>0)&(PAIB.B>0)]
    return _PAIB.freq.sum()

In [87]:
# Co-occurrence
for i in range(0,len(item_item_matrix.columns)):
    for j in range(0,len(item_item_matrix.columns)):
        if i != j:
            item_item_matrix.iloc[i,j] = cnt_Grpby(join_df,i,j)
        else:
            item_item_matrix.iloc[i,j] = 0

In [88]:
item_item_matrix.head()

,เมจิ,ไทยเดนมาร์ค,ดัชมิลล์,แมคโนเลีย,จิตรลดา,แดรี่โฮม,โฟร์โมสต์,โชคชัย,คาเนชั่น,หนองโพ,...,พนักงานขายนม,ความแข็งแรง,ผิดสังเกต,เสียความรู้สึก,ไอโอดีน,พรีเซ็นเตอร์,สุนัข,คายทิ้ง,มีลูก,คุณแม่มือใหม่
เมจิ,0,77,98,4,42,33,123,62,12,78,...,1,0,1,1,0,3,0,0,0,0
ไทยเดนมาร์ค,77,0,26,1,87,22,83,45,0,104,...,0,1,0,0,0,0,0,0,5,0
ดัชมิลล์,98,26,0,1,21,5,64,22,1,30,...,0,1,0,1,0,1,0,1,1,0
แมคโนเลีย,4,1,1,0,1,1,3,1,0,0,...,0,0,0,0,0,0,0,0,0,0
จิตรลดา,42,87,21,1,0,11,39,30,0,75,...,0,1,0,0,0,0,0,0,3,0


In [89]:
item_item_matrix.to_excel(comment_cooc_freq_file, index=True)

### Create co-occurrence matrix with Lift normalization

In [90]:
item_item_matrix = pd.DataFrame(index=join_df.columns,columns=join_df.columns).fillna(0)

In [91]:
def a_in_b(df,i,j):
    PAIB = pd.DataFrame(df.groupby(df.iloc[:,j].name)[df.iloc[:,i].name].value_counts())
    PAIB.index.names = ['B','A']
    PAIB.columns = ['freq']
    PAIB.reset_index(inplace=True)
    PAIB_ = PAIB[(PAIB.A>0)&(PAIB.B>0)]
    return PAIB_.freq.sum()/PAIB.freq.sum()

In [92]:
for i in range(0,len(item_item_matrix.columns)) :
    for j in range(0,len(item_item_matrix.columns)) :
        A = join_df.iloc[:,i]
        B = join_df.iloc[:,j]
        PA = A[A!=0].count()/A.shape[0]
        PB = B[B!=0].count()/B.shape[0]
        PAB = PB*a_in_b(join_df,i,j)
        item_item_matrix.iloc[i,j] = PAB/(PA*PB)

In [93]:
item_item_matrix.replace([np.inf, -np.inf], np.nan,inplace=True)
item_item_matrix.fillna(0,inplace=True)

In [94]:
item_item_matrix.head()

,เมจิ,ไทยเดนมาร์ค,ดัชมิลล์,แมคโนเลีย,จิตรลดา,แดรี่โฮม,โฟร์โมสต์,โชคชัย,คาเนชั่น,หนองโพ,...,พนักงานขายนม,ความแข็งแรง,ผิดสังเกต,เสียความรู้สึก,ไอโอดีน,พรีเซ็นเตอร์,สุนัข,คายทิ้ง,มีลูก,คุณแม่มือใหม่
เมจิ,1.000000,0.049170,0.062580,0.002554,0.026820,0.021073,0.078544,0.039591,0.007663,0.049808,...,0.000639,0.000000,0.000639,0.000639,0.0,0.001916,0.0,0.000000,0.000000,0.0
ไทยเดนมาร์ค,0.165236,1.000000,0.055794,0.002146,0.186695,0.047210,0.178112,0.096567,0.000000,0.223176,...,0.000000,0.002146,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.010730,0.0
ดัชมิลล์,0.270718,0.071823,1.000000,0.002762,0.058011,0.013812,0.176796,0.060773,0.002762,0.082873,...,0.000000,0.002762,0.000000,0.002762,0.0,0.002762,0.0,0.002762,0.002762,0.0
แมคโนเลีย,0.444444,0.111111,0.111111,1.000000,0.111111,0.111111,0.333333,0.111111,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0
จิตรลดา,0.132075,0.273585,0.066038,0.003145,1.000000,0.034591,0.122642,0.094340,0.000000,0.235849,...,0.000000,0.003145,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.009434,0.0


In [95]:
item_item_matrix.to_excel(comment_cooc_lift_file, index=True)

### Create co-occurrence matrix with Jaccard Distance<br>
* สำหรับทำ Robusness check

In [96]:
item_item_matrix = pd.DataFrame(index=join_df.columns,columns=join_df.columns).fillna(0)

In [97]:
def jaccard_sim(df,i,j):
    PAIB = pd.DataFrame(df.groupby(df.iloc[:,i].name)[df.iloc[:,j].name].value_counts())
    PAIB.index.names = ['A','B']
    PAIB.columns = ['freq']
    PAIB.reset_index(inplace=True)
    return jaccard_score(PAIB.A, PAIB.B, average='macro')

In [98]:
# Co-occurrence
for i in range(0,len(item_item_matrix.columns)):
    for j in range(0,len(item_item_matrix.columns)):
        if i != j:
            item_item_matrix.iloc[i,j] = jaccard_sim(join_df,i,j)
        else:
            item_item_matrix.iloc[i,j] = 0

In [99]:
item_item_matrix.head()

,เมจิ,ไทยเดนมาร์ค,ดัชมิลล์,แมคโนเลีย,จิตรลดา,แดรี่โฮม,โฟร์โมสต์,โชคชัย,คาเนชั่น,หนองโพ,...,พนักงานขายนม,ความแข็งแรง,ผิดสังเกต,เสียความรู้สึก,ไอโอดีน,พรีเซ็นเตอร์,สุนัข,คายทิ้ง,มีลูก,คุณแม่มือใหม่
เมจิ,0.000000,0.036851,0.024578,0.037879,0.019048,0.022222,0.029208,0.042424,0.047902,0.039394,...,0.053719,0.007576,0.007576,0.006993,0.004902,0.030303,0.006993,0.007576,0.006061,0.007576
ไทยเดนมาร์ค,0.036851,0.000000,0.084416,0.079365,0.108333,0.060606,0.047078,0.050000,0.018519,0.108995,...,0.023810,0.023810,0.020833,0.020833,0.011905,0.023810,0.020833,0.023810,0.044444,0.023810
ดัชมิลล์,0.024578,0.084416,0.000000,0.065476,0.036281,0.047619,0.040873,0.041558,0.061905,0.050000,...,0.017857,0.017857,0.015873,0.063492,0.010989,0.065476,0.015873,0.065476,0.012987,0.017857
แมคโนเลีย,0.037879,0.079365,0.065476,0.000000,0.100000,0.079365,0.065476,0.100000,0.050000,0.033333,...,0.166667,0.166667,0.083333,0.083333,0.017857,0.166667,0.083333,0.166667,0.033333,0.166667
จิตรลดา,0.019048,0.108333,0.036281,0.100000,0.000000,0.058333,0.079365,0.055556,0.025000,0.155556,...,0.033333,0.033333,0.028571,0.028571,0.012987,0.033333,0.028571,0.033333,0.072222,0.033333


In [100]:
item_item_matrix.to_excel(comment_cooc_jacc_file, index=True)

### Robustness check